# Tutorial #2: Experiment and train models using features

In this tutorial series you will experience how features seamlessly integrates all the phases of ML lifecycle: Prototyping features, training and operationalizing.

In part 1 of the tutorial you learnt how to create a feature set spec with custom transformations, enable materialization and perform backfill. In this tutorial you will will learn how to experiment with features to improve model performance. You will see how feature store increasing agility in the experimentation and training flows. 

You will perform the following:
- Prototype a create new `acccounts` feature set spec using existing precomputed values as features, unlike part 1 of the tutorial where we created feature set that had custom transformations. You will then Register the local feature set spec as a feature set in the feature store
- Select features for the model: You will select features from the `transactions` and `accounts` feature sets and save them as a feature-retrieval spec
- Run training pipeline that uses the Feature retrieval spec to train a new model. This pipeline will use the built in feature-retrieval component to generate the training data

# Prerequisites
1. Please ensure you have executed `1. Develop a feature set and register with managed feature store` tutorial

# Setup

#### (updated)  Configure Azure ML spark notebook

1. Running the tutorial: You can either create a new notebook, and execute the instructions in this document step by step or open the existing notebook named `Experiment and train models using features`, and run it. The notebooks are available in `featurestore_sample/notebooks` directory. You can select from `sdk_only` or `sdk_and_cli`. You may keep this document open and refer to it for additional explanation and documentation links.
1. In the "Compute" dropdown in the top nav, select "Serverless Spark Compute". 
1. Click on "configure session" in top status bar -> click on "Python packages" -> click on "upload conda file" -> select the file azureml-examples/sdk/python/featurestore-sample/project/env/conda.yml from your local machine; Also increase the session time out (idle time) if you want to avoid running the prerequisites frequently


#### Start spark session

In [ ]:
# run this cell to start the spark session (any code block will start the session ). This can take around 10 mins.
print("start spark session")

#### Setup root directory for the samples

In [ ]:
import os

# please update the dir to ./Users/<your_user_alias> (or any custom directory you uploaded the samples to).
# You can find the name from the directory structure inm the left nav
root_dir = "./Users/<your_user_alias>/featurestore_sample"

if os.path.isdir(root_dir):
    print("The folder exists.")
else:
    print("The folder does not exist. Please create or fix the path")

#### (new for sdk/cki track) Setup CLI

1. Install azure ml cli extention
1. Authenticate
1. Set the default subscription

In [ ]:
!az extension add --name ml

In [ ]:
!az login

In [ ]:
import os

subscription_id = os.environ["AZUREML_ARM_SUBSCRIPTION"]

!az account set -s $subscription_id

#### Initialize the project workspace variables
This is the current workspace where you will be running the tutorial notebook from

In [ ]:
project_ws_sub_id = os.environ["AZUREML_ARM_SUBSCRIPTION"]
project_ws_rg = os.environ["AZUREML_ARM_RESOURCEGROUP"]
project_ws_name = os.environ["AZUREML_ARM_WORKSPACE_NAME"]

#### Initialize the feature store variables
Ensure you update the `featurestore_name` to reflect what you created in part 1 of this tutorial

In [ ]:
featurestore_name = (
    "<FEATURESTORE_NAME>"  # use the same name from part #1 of the tutorial
)
featurestore_subscription_id = os.environ["AZUREML_ARM_SUBSCRIPTION"]
featurestore_resource_group_name = os.environ["AZUREML_ARM_RESOURCEGROUP"]

#### Initialize the feature store consumption client

In [ ]:
# feature store client
from azureml.featurestore import FeatureStoreClient
from azure.ai.ml.identity import AzureMLOnBehalfOfCredential

featurestore = FeatureStoreClient(
    credential=AzureMLOnBehalfOfCredential(),
    subscription_id=featurestore_subscription_id,
    resource_group_name=featurestore_resource_group_name,
    name=featurestore_name,
)

#### Create compute cluster with name `cpu-cluster-fs` in the project workspace
This will be needed when we run training/batch inference jobs

In [ ]:
compute_cluster_name = "<COMPUTE_CLUSTER_NAME>"
type = "<COMPUTE_TYPE>"
size = "<COMPUTE_SIZE>"

In [ ]:
!az ml compute create --name $compute_cluster_name --type $type --size $size --idle-time-before-scale-down 360 --resource-group $project_ws_rg --workspace-name $project_ws_name

## Step 1: Create accounts featureset locally from precomputed data
In tutorial part 1, we created a transactions featureset that had custom transformations. Now we will create an accounts featureset that will use precomputed values. 

For onboarding precomputed features, you can create a featureset spec without writing any transformation code. Featureset spec is a specification to develop and test a featureset in a fully local/dev environment without connecting to any featurestore. In this step you will create the feature set spec locally and sample the values from it. If you want to get managed featurestore capabilities, you need to register the featureset spec with a feature store using a feature asset definition (a future step in this tutorial).

#### Step 1a: Explore the source data for accounts

##### Note
 Note that the sample data used in this notebook is hosted in a public accessible blob container. It can only be read in Spark via `wasbs` driver. When you create feature sets using your own source data, please host them in adls gen2 account and use `abfss` driver in the data path.  

In [ ]:
accounts_data_path = "wasbs://data@azuremlexampledata.blob.core.windows.net/feature-store-prp/datasources/accounts-precalculated/*.parquet"
accounts_df = spark.read.parquet(accounts_data_path)

display(accounts_df.head(5))

#### Step 1b: Create `accounts` feature set spec in local from these precomputed features
Note that we do not need any transformation code here since we are referencing precomputed features.

In [ ]:
from azureml.featurestore import create_feature_set_spec, FeatureSetSpec
from azureml.featurestore.contracts import (
    DateTimeOffset,
    FeatureSource,
    TransformationCode,
    Column,
    ColumnType,
    SourceType,
    TimestampColumn,
)


accounts_featureset_spec = create_feature_set_spec(
    source=FeatureSource(
        type=SourceType.parquet,
        path="wasbs://data@azuremlexampledata.blob.core.windows.net/feature-store-prp/datasources/accounts-precalculated/*.parquet",
        timestamp_column=TimestampColumn(name="timestamp"),
    ),
    index_columns=[Column(name="accountID", type=ColumnType.string)],
    infer_schema=True,
)

In [ ]:
# Generate a spark dataframe from the feature set specification
accounts_fset_df = accounts_featureset_spec.to_spark_dataframe()
# display few records
display(accounts_fset_df.head(5))

#### Step 1c:  Export as feature set spec
Inorder to register the feature set spec with the feature store, it needs to be saved in a specific format. 
Action: After running the below cell, please inspect the generated `accounts` FeatureSetSpec: Open this file from the file tree to see the spec: `featurestore/featuresets/accounts/spec/FeatureSetSpec.yaml`

Spec contains these important elements:

1. `source`: reference to a storage. In this case a parquet file in a blob storage.
1. `features`: list of features and their datatypes. If you provide transformation code (see Day 2 section), the code has to return a dataframe that maps to the features and datatypes. In case where you do not provide transformation code (in this case of `accounts` because it is precomputed), the system will build the query to to map these to the source 
1. `index_columns`: the join keys required to access values from the feature set

Learn more about it in the [top level feature store entities document](fs-concepts-todo) and the [feature set spec yaml reference](reference-yaml-featureset-spec.md).

The additional benefit of persisting it is that it can be source controlled.

In [ ]:
import os

# create a new folder to dump the feature set spec
accounts_featureset_spec_folder = root_dir + "/featurestore/featuresets/accounts/spec"

# check if the folder exists, create one if not
if not os.path.exists(accounts_featureset_spec_folder):
    os.makedirs(accounts_featureset_spec_folder)

accounts_featureset_spec.dump(accounts_featureset_spec_folder, overwrite=True)

## Step 2: Experiment with unregistered features locally and register with feature store when ready
When you are developing features, you might want to test/validate locally before registering with the feature store or running training pipelines in the cloud. In this step you will generate training data for the ML model from combination of features from a local unregistered feature set (`accounts`) and feature set registered in the feature store (`transactions`).

#### Step 2a: Select features for model

In [ ]:
# get the registered transactions feature set, version 1
transactions_featureset = featurestore.feature_sets.get("transactions", "1")
# Notice that account feature set spec is in your local dev environment (this notebook): not registered with feature store yet
features = [
    accounts_featureset_spec.get_feature("accountAge"),
    accounts_featureset_spec.get_feature("numPaymentRejects1dPerUser"),
    transactions_featureset.get_feature("transaction_amount_7d_sum"),
    transactions_featureset.get_feature("transaction_amount_3d_sum"),
    transactions_featureset.get_feature("transaction_amount_7d_avg"),
]

#### Step 2b: Generate training data locally
In this step we generate training data for illustrative purpose. You can optionally train models locally with this. In the upcoming steps in this tutorial, you will train a model in the cloud.

In [ ]:
from azureml.featurestore import get_offline_features

# Load the observation data. To understand observatio ndata, refer to part 1 of this tutorial
observation_data_path = "wasbs://data@azuremlexampledata.blob.core.windows.net/feature-store-prp/observation_data/train/*.parquet"
observation_data_df = spark.read.parquet(observation_data_path)
obs_data_timestamp_column = "timestamp"

In [ ]:
# generate training dataframe by using feature data and observation data
training_df = get_offline_features(
    features=features,
    observation_data=observation_data_df,
    timestamp_column=obs_data_timestamp_column,
)

# Ignore the message that says feature set is not materialized (materialization is optional). We will enable materialization in the next part of the tutorial.
display(training_df)
# Note: display(training_df.head(5)) displays the timestamp column in a different format. You can can call training_df.show() to see correctly formatted value

#### Step 2c: Register the `accounts` featureset with the featurestore
Once you have experimented with different feature definitions locally and sanity tested it, you can register it with the feature store.
For this you will register a featureset asset definition with the feature store.


In [ ]:
feature_set_yaml = root_dir + "/featurestore/featuresets/accounts/featureset_asset.yaml"

!az ml feature-set create --file $feature_set_yaml --resource-group $featurestore_resource_group_name --feature-store-name $featurestore_name

#### Step 2d: Get registered featureset and sanity test

In [ ]:
# look up the featureset by providing name and version
accounts_featureset = featurestore.feature_sets.get("accounts", "1")

In [ ]:
# get access to the feature data
accounts_feature_df = accounts_featureset.to_spark_dataframe()
display(accounts_feature_df.head(5))
# Note: Please ignore this warning: Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun

## Step 3: Run training experiment
In this step you will select a list of features, run a training pipeline, and register the model. You can repeat this step till you are happy with the model performance.

#### (Optional) Step 3a: Discover features from Feature Store UI
You have already done this in part 1 of the tutorial after registering the `transactions` feature set. Since you also have `accounts` featureset, you can browse the available features:
* Goto the [Azure ML global landing page](https://ml.azure.com/home?flight=FeatureStores).
* Click on `Feature stores` in the left nav
* You will see the list of feature stores that you have access to. Click on the feature store that you created above.

You can see the feature sets and entity that you created. Click on the feature sets to browse the feature definitions. You can also search for feature  sets across feature stores by using the global search box.

#### (Optional) Step 3b: Discover features from SDK

In [ ]:
# List available feature sets
all_featuresets = featurestore.feature_sets.list()
for fs in all_featuresets:
    print(fs)

# List of versions for transactions feature set
all_transactions_featureset_versions = featurestore.feature_sets.list(
    name="transactions"
)
for fs in all_transactions_featureset_versions:
    print(fs)

# See properties of the transactions featureset including list of features
featurestore.feature_sets.get(name="transactions", version="1").features

#### Step 3c: Select features for the model and export it as a feature-retrieval spec
In the previous steps, you selected features from a combination unregistered  and registered feature sets for local experimentation and testing. Now you are ready to experiment in the cloud. Saving the selected features as a feature-retrieval spec and using it in the mlops/cicd flow for training/inference increases your agility in shipping models.

Select features for the model

In [ ]:
# you can select features in pythonic way
features = [
    accounts_featureset.get_feature("accountAge"),
    transactions_featureset.get_feature("transaction_amount_7d_sum"),
    transactions_featureset.get_feature("transaction_amount_3d_sum"),
]

# you can also specify features in string form: featurestore:featureset:version:feature
more_features = [
    "accounts:1:numPaymentRejects1dPerUser",
    "transactions:1:transaction_amount_7d_avg",
]

more_features = featurestore.resolve_feature_uri(more_features)

features.extend(more_features)

Export selected features as a feature-retrieval spec

#### Note
Feature retrieval spec is a portable definition of list of features associated with a model. This can help streamline ML model development and operationalizing.This will be an input to the training pipeline (used to generate the training data), then will be packaged along with the model, and will be used during inference to lookup the features. It will be a glue that integrates all phases of the ML lifecycle. Changes to training/inference pipeline can be kept minimal as you experiment and deploy. 

Using feature retrieval spec and the built-in feature retrieval component is optional: you can directly use `get_offline_features()` api as shown above.

Note that the name of the spec should be `feature_retrieval_spec.yaml` when it is packaged with the model for the system to recognize it.

In [ ]:
# Create feature retrieval spec
feature_retrieval_spec_folder = root_dir + "/project/fraud_model/feature_retrieval_spec"

# check if the folder exists, create one if not
if not os.path.exists(feature_retrieval_spec_folder):
    os.makedirs(feature_retrieval_spec_folder)

featurestore.generate_feature_retrieval_spec(feature_retrieval_spec_folder, features)

## Step 4: Train in the cloud using pipelines and register model if satisfactory
In this step you will manually trigger the training pipeline. In a production scenario, this could be triggered by a ci/cd pipeline based on changes to the feature-retrieval spec in the source repository.

#### Step 4a: Run the training pipeline
The training pipeline has the following steps:

1. Feature retrieval step: This is a built-in component takes as input the feature retrieval spec, the observation data and timestamp column name. It then generates the training data as output. It runs this as a managed spark job.
1. Training step: This step trains the model based on the training data and generates a model (not registered yet)
1. Evaluation step: This step validates whether model performance/quailty is within threshold (in our case it is a placeholder/dummy step for illustration purpose)
1. Register model step: This step registers the model

Note: In part 2 of this tutorial you ran a backfill job to materialize data for `transactions` feature set. Feature retrieval step will read feature values from offline store for this feature set. The behavior will same even if you use `get_offline_features()` api.

In [ ]:
training_pipeline_path = (
    root_dir + "/project/fraud_model/pipelines/training_pipeline.yaml"
)

!az ml job create --file $training_pipeline_path --resource-group $project_ws_rg --workspace-name $project_ws_name
# Note: First time it runs, each step in pipeline can take ~ 15 mins. However subsequent runs can be faster (assuming spark pool is warm - default timeout is 30 mins)

#### Inspect the training pipeline and the model
Open the above pipeline run "web view" in new window to see the steps in the pipeline.


#### Step 4b: Notice the feature retrieval spec in the model artifacts
1. In the left nav of the current workspace -> right click on `Models` -> Open in new tab or window
1. Click on `fraud_model`
1. Click on `Artifacts` in the top nav

You can notice that the feature retrieval spec is packaged along with the model. The model registration step in the training pipeline has done this. You created feature retrieval spec during experimentation, now it has become part of the model definition. In the next tutorial you will see how this will be used during inferencing.


## Step 5: View the feature set and model dependencies

#### Step 5a: View the list of feature sets associated with the model
In the same models page, click on the `feature sets` tab. Here you can see both `transactions` and `accounts` featuresets that this model depends on.

#### Step 5b: View the list of models using the feature sets
1. Open the feature store UI (expalined in a previous step in this tutorial)
1. Click on `Feature sets` on the left nav
1. Click on any of the feature set -> click on `Models` tab

You can see the list of models that are using the feature sets (determined from the feature retrieval spec when the model was registered).

## Cleanup

Tutorial of `Enable recurrent materialization and run batch inference` has instructions for deleting the resources

## Next steps
* Enable recurrent materialization and run batch inference